In [1]:
import json
from pathlib import Path
from typing import List, Tuple
from tqdm import tqdm

In [7]:
clean_path = Path("tc_data/filtered_Ewo/MAT/MAT")
json_path = Path("/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/tc_data/MAT.json")
book, clean_verset = json_path.stem, clean_path.stem

In [39]:
clean_path = Path("/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/tc_data/filtered_Ewo")
txt_path= sorted(clean_path.rglob("*.txt"))

In [21]:
print(f'book: {book} chap_clean: {clean_verset}')

book: MAT chap_clean: MAT_001_001


In [9]:
get_chapter = lambda x: x.split('.')[0] + '_' + x.split('.')[1].zfill(3) + '_'+ x.split('.')[-1].zfill(3)


In [24]:
k= get_chapter("MAT.1.1")

In [25]:
k

'MAT_001_001'

In [2]:
from norm_cf import norm_conf
from text_normalization import text_normalize

In [3]:
transcript_o='Adam a too mimbu 130, a nga biaé monafam a funaʼane nye, aval éyekaʼa jé é né, a a nga yôlé nye jôé na Sét.'

In [4]:
transcript = text_normalize(transcript_o, 'bum',remove_numbers=False)

In [5]:
print(f'original transcript:\n{transcript_o}')
print(f'normalized transcript:\n{transcript}')

original transcript:
Adam a too mimbu 130, a nga biaé monafam a funaʼane nye, aval éyekaʼa jé é né, a a nga yôlé nye jôé na Sét.
normalized transcript:
adam a too mimbu * a nga biaé monafam a funaʼane nye aval éyekaʼa jé é né a a nga yôlé nye jôé na sét


In [64]:
transcript

'adam a too mimbu a nga biaé monafam a funaʼane nye aval éyekaʼa jé é né a a nga yôlé nye jôé na sét'

In [8]:
def load_transcripts(json_path: Path, clean_verset: str) -> Tuple[List[str], List[str]]:
    with open(json_path, "r") as f:
        data = json.load(f)
        #print(data)

    # convert MAT.19.1 -> MAT_019_001
    get_chapter = lambda x: x.split('.')[0] + '_' + x.split('.')[1].zfill(3) + '_'+ x.split('.')[-1].zfill(3)

    transcripts = [d["verset"] for d in data if get_chapter(d["numVerset"]) == clean_verset]
    verse_ids = [d["numVerset"] for d in data if get_chapter(d["numVerset"]) == clean_verset]
    return verse_ids, transcripts

In [19]:


# Assurez-vous que json_path est défini avant l'exécution
clean_path = Path("/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/tc_data/filtered_Ewo/MAT/MAT/MAT_001")
txt_paths = sorted(clean_path.rglob("*.txt"))

for txt_path in tqdm(txt_paths, desc="Processing"):
    verset_c = txt_path.stem

    # Chargement du transcript depuis le JSON
    try:
        _, transcript_o = load_transcripts(json_path, verset_c)
    except Exception as e:
        print(f"Error loading transcript for {verset_c}: {e}")
        continue

    # Lecture du fichier texte
    try:
        with open(txt_path, 'r', encoding='utf-8') as f:
            transcript_c = f.read().strip()
    except Exception as e:
        print(f"Error reading file {txt_path}: {e}")
        continue

    # Normalisation et écriture du fichier
    try:
        transcript_norm = text_normalize(transcript_o[0], 'deu')

        # Écriture dans le fichier (remplacement de l'ancien contenu)
        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write(transcript_norm)

    except Exception as e:
        print(f"Problème de remplacement et/ou de normalisation pour {verset_c}: {e}")
        continue


Processing:  17%|█▋        | 4/24 [00:00<00:00, 39.57it/s]

Processing: 100%|██████████| 24/24 [00:00<00:00, 38.11it/s]


In [ ]:
def build_manifest(file:Path, outdir:str):

    txt_paths= sorted(clean_path.rglob("*.txt"))
    audios   =sorted(clean_path.rglob("*.wav"))
    for txt_path,audio_path in zip(tqdm(txt_paths, desc=f"processing"),audios):
        verset_c= txt_path.stem
        

        try:
            with open(txt_path,'r') as f:
                norm_transcripts= f.read().strip()
                tokens= get_uroman_tokens(norm_transcripts,'bum','Bulu')
        except Exception as e:
            print(f"Error reading file {txt_path}: {e}")
        continue

        with open( f"{outdir}/manifest.json", "w+") as f:
            sample = {
            
                "audio_filepath": str(audio_path),
                "text": text,
                "normalized_text":norm_transcripts[i],
                "uroman_tokens": tokens,
            }
            f.write(json.dumps(sample) + "\n")

        

In [29]:
verse_ids, transcripts = load_transcripts(json_path, clean_verset)

[{'numVerset': 'MAT.1.1', 'verset': 'Kalara  mvòṅ bod Yesus Kristus, man David, man Abraham. '}, {'numVerset': 'MAT.1.2', 'verset': "Abraham abye Isaak, Isaak ny'abye Yakòb, Yakòb abye Yuda ai babënyaṅ, "}, {'numVerset': 'MAT.1.3', 'verset': 'Yuda abye Farès ban Zara, ai Tamar, Farès abye Esrom, Esrom abye Aram, '}, {'numVerset': 'MAT.1.4', 'verset': 'Aram abye Aminadab, Aminadab abye Naason, Naason abye Salmon, '}, {'numVerset': 'MAT.1.5', 'verset': 'Salmon abye Boaz ai Rahab, Boaz abye Yobèd ai Ruth, Yobèd abye Yesse, '}, {'numVerset': 'MAT.1.6', 'verset': 'Yesse nyè abye nkukuma David. David abye Salomon ai mininga Uria, '}, {'numVerset': 'MAT.1.7', 'verset': 'Salomon abye Roboam, Roboam abye Abia, Abia abye Asaf. '}, {'numVerset': 'MAT.1.8', 'verset': 'Asaf abye Yosafat, Yosafat abye Yoram, Yoram abye Ozia, '}, {'numVerset': 'MAT.1.9', 'verset': 'Ozia abye Yoatam, Yoatam abye Akas, Akas abye Ezekias, '}, {'numVerset': 'MAT.1.10', 'verset': 'Ezekias abye Manasse, Manasse abye Amos, 

In [30]:
transcripts

['Kalara  mvòṅ bod Yesus Kristus, man David, man Abraham. ']

In [31]:
verse_ids

['MAT.1.1']

In [4]:
from norm_cf import norm_conf

In [ ]:
import os
from tqdm import tqdm

# Dossier où se trouvent les fichiers compressés
dossier_compression = '/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/Compressed_Books-20250210T214758Z-001/Compressed_Books'
# Dossier où les fichiers seront décompressés
dossier_decompression = 'ewondo_leo'
os.makedirs(dossier_decompression, exist_ok=True)

# Fonction pour décompresser un fichier .tar.zst avec une barre de progression
def decompresser_tar_zst(fichier_tar_zst):
    chemin_fichier = os.path.join(dossier_compression, fichier_tar_zst)
    
    with tqdm(total=1, desc=f'Décompression de {fichier_tar_zst}', unit='fichier') as pbar:
        # Commande pour extraire l'archive en conservant la structure des dossiers
        os.system(f'tar -I zstd -xvf "{chemin_fichier}" -C "{dossier_decompression}"')
        pbar.update(1)
    
    print(f"{fichier_tar_zst} décompressé avec succès!")

# Lister et décompresser tous les fichiers .tar.zst avec une barre de progression globale
fichiers_zst = [fichier for fichier in os.listdir(dossier_compression) if fichier.endswith('.tar.zst')]

for fichier in tqdm(fichiers_zst, desc="Décompression des livres", unit="livre"):
    decompresser_tar_zst(fichier)


## Test romanisation

In [23]:

import uroman as ur

In [24]:
# load uroman data (once at the beginning)
uroman = ur.Uroman()

In [25]:

for s in ('Νεπάλ', 'नेपाल', 'نیپال', '三万一'):
    print(s, uroman.romanize_string(s))

Νεπάλ Nepal
नेपाल nepaal
نیپال nipal
三万一 31000


In [26]:
txt='Mɛ̀ Pol ànɛ ǹɗì ǹɗeŋiyɛn lɛ mɓa ǹkɛ̀nì-fìtom a Yeesus Kiris lɛ rìkɔ̀n ɗi Bɛ̀ll, rì Sostɛn à geni wìs.'
print(uroman.romanize_string(txt))

Me Pol ane ndi ndengiyen le mba nkeni-fitom a Yeesus Kiris le rikon di Bell, ri Sosten a geni wis.


In [46]:
import re
import uroman as ur
from tc_utils import normalize_uroman
import os
import tempfile
from tqdm import tqdm

special_isos_uroman = "ara, bel, bul, deu, ell, eng, fas, grc, ell, eng, heb, kaz, kir, lav, lit, mkd, mkd2, oss, pnt, pus, rus, srp, srp2, tur, uig, ukr, yid".split(",")
special_isos_uroman = [i.strip() for i in special_isos_uroman]



def get_uroman_tokens(norm_transcripts, iso=None):
    # load uroman data (once at the beginning)
    uroman = ur.Uroman()
    uromans = []
    
    # Création des fichiers temporaires
    with tempfile.NamedTemporaryFile(mode='w+', delete=False) as input_file, \
         tempfile.NamedTemporaryFile(mode='w+', delete=False) as output_file:

        try:
            # Écriture des textes normalisés dans le fichier temporaire d'entrée
            for text in norm_transcripts:
                input_file.write(text + "\n")
            input_file.flush()

            # Lecture du fichier et traitement avec la barre de progression
            with open(input_file.name, 'r') as infile:
                for line in tqdm(infile, desc="Romanisation des textes", unit="texte"):
                    text = line.strip()
                    try:
                        # Vérification de la présence du code ISO et romanisation
                        if iso in special_isos_uroman:
                            romanized_text = uroman.romanize_string(text,lcode=iso)
                        else:
                            romanized_text = uroman.romanize_string(text)
                        
                        # Nettoyage des textes romanisés
                        romanized_text = " ".join(romanized_text.strip())
                        romanized_text = re.sub(r"\s+", " ", romanized_text).strip()

                        # Application d'une normalisation supplémentaire
                        normalized_text = normalize_uroman(romanized_text)
                        uromans.append(normalized_text)
                    
                    except Exception as e:
                        print(f"Erreur lors de la romanisation du texte : {text}\nDétail de l'erreur : {e}")

            # Écriture des résultats dans le fichier temporaire de sortie
            with open(output_file.name, 'w') as outfile:
                for uroman_text in uromans:
                    outfile.write(uroman_text + "\n")
        
        except Exception as e:
            print(f"Erreur lors du traitement des fichiers : {e}")
        
        finally:
            # Nettoyage des fichiers temporaires
            os.remove(input_file.name)
            os.remove(output_file.name)

    return uromans



In [47]:
t_path='/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/ztext_cleaner/text.txt'
uroman_path=''
lang=''

with open(t_path,'r') as f:
    transcripts = [line.strip() for line in f]
#print("Read {} lines from {}".format(len(transcripts), t_path))

norm_transcripts = [text_normalize(line.strip(),'ksf') for line in transcripts]

tokens = get_uroman_tokens(norm_transcripts)

Romanisation des textes: 436texte [00:13, 31.93texte/s]


In [48]:
for tok in tokens[:5]:
    print(f'{tok}\n')

m e p o l a n e n d i n d e n g i y e n l e m b a n k e n i f i t o m a y e e s u s k i r i s l e r i k o n d i b e l l r i s o s t e n a g e n i w i s

t i k a n i k i s e n g i k i b e l l k i k i r i a k o r e n g t r i b u m b e b e r i b e t e n e n b i y e e s u s k i r i s b i d u m b o y e e r i b e r e n g i y e n l e b e e b a b e g h e r e n r i b u m b e c e m b e b e e r e r e n g i d i i d i k a n w i s y e e s u s k i r i s b a a s o o r i f e l e y e e s u s a r i k a n w a a n y i t i k a n w i s

b e l l a s e n g i w i s b e e k a n y e e s u s k i r i s b e e f a m i n i l e r i g e e s e n r i f y o o o g

m e e n f a f a b e l l b i t o g s e n k e e n f o y b i s u w i n l e g e e s e n i b e l l i n e i f a l e n g a a l o n g k i n b i y e e s u s k i r i s

r i t i l e b i k i r i s b i m e t e d b e m a a g h b e c e m a b i i s e n b e b e s e e s e e k i t o k i r i r i y i d i b i b a n



#### test with iso code

In [42]:


t_path='/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/ztext_cleaner/text.txt'

lang='deu'

with open(t_path,'r') as f:
    transcripts = [line.strip() for line in f]
#print("Read {} lines from {}".format(len(transcripts), t_path))

norm_transcripts = [text_normalize(line.strip(),'deu') for line in transcripts]

tokens = get_uroman_tokens(norm_transcripts)

In [43]:
for tok in tokens[:5]:
    print(f'{tok}\n')

m e p o l a n e n d i n d e n g i y e n l e m b a n k e n i f i t o m a y e e s u s k i r i s l e r i k o n d i b e l l r i s o s t e n a g e n i w i s

t i k a n i k i s e n g i k i b e l l k i k i r i a k o r e n g t r i b u m b e b e r i b e t e n e n b i y e e s u s k i r i s b i d u m b o y e e r i b e r e n g i y e n l e b e e b a b e g h e r e n r i b u m b e c e m b e b e e r e r e n g i d i i d i k a n w i s y e e s u s k i r i s b a a s o o r i f e l e y e e s u s a r i k a n w a a n y i t i k a n w i s

b e l l a s e n g i w i s b e e k a n y e e s u s k i r i s b e e f a m i n i l e r i g e e s e n r i f y o o o g

m e e n f a f a b e l l b i t o g s e n k e e n f o y b i s u w i n l e g e e s e n i b e l l i n e i f a l e n g a a l o n g k i n b i y e e s u s k i r i s

r i t i l e b i k i r i s b i m e t e d b e m a a g h b e c e m a b i i s e n b e b e s e e s e e k i t o k i r i r i y i d i b i b a n



In [34]:
s = 'Игорь'
for lang_code in ('rus', 'ukr', None):
    print(s, lang_code, uroman.romanize_string(s, lcode=lang_code))

Игорь rus Igor
Игорь ukr Yhor
Игорь None Igor
